# Public Housing & Building Layer
As they are too heavy to handle in R, I will use Python.

## Reading Important Modules

In [1]:
import geopandas as gpd

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Reading Files

In [2]:
ph = gpd.read_file(r"/Volumes/volume/GIS Projects/nightlight/Public_Housing_Buildings.geojson")

In [3]:
boundary = gpd.read_file("/Volumes/volume/GIS Projects/nightlight/phoenix/city_boundary/City_Limit_Dark_Outline.shp")

In [4]:
building = gpd.read_file("/Volumes/volume/GIS Projects/nightlight/phoenix/LiDAR-Derived_2D_Building_Footprints_-_metro-Phoenix%2C_Arizona_(2014).geojson")

In [5]:
grid = gpd.read_file("/Users/naoyamorishita/Documents/working/nightlight/data/grid.geojson")

## Manipulating Public Housing Layer
I wanted to clip the public housing layer by the city boundary. However, the CRS might not be the same. Hence, I will align their CRS, before clipping the layer.

In [6]:
ph = ph.to_crs(boundary.crs)

In [7]:
ph.crs

<Derived Projected CRS: EPSG:2868>
Name: NAD83(HARN) / Arizona Central (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Arizona - counties Coconino; Maricopa; Pima; Pinal; Santa Cruz; Yavapai.
- bounds: (-113.35, 31.33, -110.44, 37.01)
Coordinate Operation:
- name: SPCS83 Arizona Central zone (International feet)
- method: Transverse Mercator
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
ph = ph.clip(boundary)

In [9]:
ph

,OBJECTID,PARTICIPANT_CODE,FORMAL_PARTICIPANT_NAME,DEVELOPMENT_CODE,PROJECT_NAME,BUILDING_NAME,BUILDING_NUMBER,BUILDING_TYPE_CODE,BUILDING_STATUS_TYPE_CODE,NATIONAL_BLDG_ID,...,ANNL_EXPNS_AMNT_PREV_YR,PHA_TOTAL_UNITS,DEV_BUIL_NU_ENTRANCE,HA_PHN_NUM,HA_FAX_NUM,HA_EMAIL_ADDR_TEXT,EXEC_DIR_PHONE,EXEC_DIR_FAX,EXEC_DIR_EMAIL,geometry
102270,102271,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,004,SF,DDAPRD,7503200000,...,-4.0,1556,AZ0010000080041,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (650397.233 861223.677)
136530,136531,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,284,SF,INAPCP,3233200000,...,-4.0,1556,AZ0010000082841,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (658707.829 861743.379)
39278,39279,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,008,SF,RMIPRP,0603200000,...,-4.0,1556,AZ0010000080081,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (650005.892 865528.232)
187137,187138,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,011,SF,INAPCP,1603200000,...,-4.0,1556,AZ0010000080111,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (656520.193 866055.328)
56296,56297,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,286,SF,INAPCP,5233200000,...,-4.0,1556,AZ0010000082861,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (679548.691 867033.110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130792,130793,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,496,SF,INAPCP,9153200000,...,-4.0,1556,AZ0010000084961,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (653470.733 969185.682)
8784,8785,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,501,SF,INAPCP,4253200000,...,-4.0,1556,AZ0010000085011,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (654193.355 969191.534)
149358,149359,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,481,SF,INAPCP,5053200000,...,-4.0,1556,AZ0010000084811,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (649708.190 970084.032)
129172,129173,AZ001,City of Phoenix Housing Department,AZ001000008,Scattered Sites AZ1-40,None,352,SF,INAPCP,6833200000,...,-4.0,1556,AZ0010000083521,6022626794,6025344516,lydia.martinez@phoenix.gov,6024956742,None,titus.mathew@phoenix.gov,POINT (634559.229 970358.975)


## Calculating the Building Density
The building layer will be used to calculate the housing density in a grid. To do this, I will calculate areas of the buildings, and aggregate the areas to sum up them within a grid. 

In [10]:
building = building.to_crs(boundary.crs) # crs transformation

In [11]:
building.head

<bound method NDFrame.head of            FID  AREA_M2  AVGHT_M  MINHT_M  MAXHT_M  BASE_M  ORIENT8    LEN  \
0            1  167.065     3.36     0.96     4.91  361.75     3.00  18.55   
1            2  217.389     3.45    -0.05     9.88  362.12    -1.10  17.91   
2            3  227.174     3.42     0.00     5.10  361.55     3.70  19.04   
3            4  278.697     3.07     0.03     4.63  361.86     0.00  17.27   
4            5  285.882     3.09     0.00     4.95  362.11    18.19  19.54   
...        ...      ...      ...      ...      ...     ...      ...    ...   
792671  792672  246.409     3.52     0.01     4.90  366.44     3.10  18.90   
792672  792673  228.340     3.68    -0.01     7.12  366.44     5.20  15.59   
792673  792674  234.845     3.75    -0.01     6.19  366.34     6.30  17.65   
792674  792675  274.080     4.38    -0.01     5.88  366.37     6.70  17.82   
792675  792676  290.546     3.79     0.00     7.31  366.23     2.70  21.52   

          WID     ID        CITY 

In [12]:
building["area"] = building["geometry"].area # calculating the area

In [13]:
building.head

<bound method NDFrame.head of            FID  AREA_M2  AVGHT_M  MINHT_M  MAXHT_M  BASE_M  ORIENT8    LEN  \
0            1  167.065     3.36     0.96     4.91  361.75     3.00  18.55   
1            2  217.389     3.45    -0.05     9.88  362.12    -1.10  17.91   
2            3  227.174     3.42     0.00     5.10  361.55     3.70  19.04   
3            4  278.697     3.07     0.03     4.63  361.86     0.00  17.27   
4            5  285.882     3.09     0.00     4.95  362.11    18.19  19.54   
...        ...      ...      ...      ...      ...     ...      ...    ...   
792671  792672  246.409     3.52     0.01     4.90  366.44     3.10  18.90   
792672  792673  228.340     3.68    -0.01     7.12  366.44     5.20  15.59   
792673  792674  234.845     3.75    -0.01     6.19  366.34     6.30  17.65   
792674  792675  274.080     4.38    -0.01     5.88  366.37     6.70  17.82   
792675  792676  290.546     3.79     0.00     7.31  366.23     2.70  21.52   

          WID     ID        CITY 

In [14]:
building = building[["ID", "geometry", "area"]]

In [15]:
building

,ID,geometry,area
0,79111,"POLYGON ((631830.483 924913.678, 631769.735 92...",1798.870966
1,79131,"POLYGON ((632276.696 924950.945, 632276.433 92...",2340.741538
2,79189,"POLYGON ((631553.076 924915.837, 631490.782 92...",2446.094926
3,79308,"POLYGON ((631963.407 924997.377, 631963.520 92...",3000.874196
4,79408,"POLYGON ((632171.307 925006.961, 632125.823 92...",3078.236950
...,...,...,...
792671,170,"POLYGON ((716023.889 812510.216, 715974.441 81...",2653.488468
792672,165,"POLYGON ((716173.847 812514.155, 716122.943 81...",2458.911336
792673,171,"POLYGON ((716215.526 812514.219, 716191.714 81...",2528.963551
792674,179,"POLYGON ((716300.648 812533.049, 716250.578 81...",2951.478136


In [16]:
grid = grid.to_crs(boundary.crs) # crs transformation
building_grid = building.sjoin(grid)

In [17]:
building_grid

,ID,geometry,area,index_right,id
0,79111,"POLYGON ((631830.483 924913.678, 631769.735 92...",1798.870966,3969,8589.0
1,79131,"POLYGON ((632276.696 924950.945, 632276.433 92...",2340.741538,3969,8589.0
2,79189,"POLYGON ((631553.076 924915.837, 631490.782 92...",2446.094926,3969,8589.0
3,79308,"POLYGON ((631963.407 924997.377, 631963.520 92...",3000.874196,3969,8589.0
4,79408,"POLYGON ((632171.307 925006.961, 632125.823 92...",3078.236950,3969,8589.0
...,...,...,...,...,...
754903,699,"POLYGON ((663897.568 834098.523, 663885.845 83...",576.186650,7481,14374.0
754904,823,"POLYGON ((663917.006 834196.286, 663888.308 83...",388.811647,7481,14374.0
754905,720,"POLYGON ((663997.810 834157.506, 663999.823 83...",8012.074482,7481,14374.0
754906,1771,"POLYGON ((663894.726 834622.077, 663867.325 83...",2686.398169,7481,14374.0


### Aggregating to Calculate the Sum

In [18]:
building_grid_agg = building_grid.groupby("id").sum("area")

In [19]:
building_grid_agg

,ID,area,index_right
id,,,
5414.0,91449,5361.799242,2213
5415.0,180578,5901.292609,4428
5419.0,1443945,27811.009286,35488
5420.0,5891684,127944.686468,144235
5421.0,7530418,137466.343330,184260
...,...,...,...
14384.0,229280,348976.091832,1026267
14385.0,347087,481938.452248,1595796
14386.0,334139,341532.921286,1266317


### Calculating the Density

In [28]:
grid["area_g"] = grid["geometry"].area
grid

,id,geometry,area_g
0,30.0,"POLYGON ((612785.880 1061225.328, 614049.904 1...",1.916528e+06
1,31.0,"POLYGON ((614049.904 1061221.815, 615313.929 1...",1.916527e+06
2,32.0,"POLYGON ((615313.929 1061218.352, 616577.954 1...",1.916526e+06
3,33.0,"POLYGON ((616577.954 1061214.942, 617841.978 1...",1.916525e+06
4,34.0,"POLYGON ((617841.978 1061211.582, 619106.002 1...",1.916524e+06
...,...,...,...
7492,14385.0,"POLYGON ((677087.327 835215.728, 678360.439 83...",1.930088e+06
7493,14386.0,"POLYGON ((678360.439 835214.839, 679633.551 83...",1.930087e+06
7494,14387.0,"POLYGON ((679633.551 835214.000, 680906.662 83...",1.930087e+06
7495,14388.0,"POLYGON ((680906.662 835213.213, 682179.774 83...",1.930087e+06


In [29]:
grid_final = building_grid_agg.merge(grid, on='id', how="right")

In [30]:
grid_final

,id,ID,area,index_right,geometry,area_g
0,30.0,NaN,NaN,NaN,"POLYGON ((612785.880 1061225.328, 614049.904 1...",1.916528e+06
1,31.0,NaN,NaN,NaN,"POLYGON ((614049.904 1061221.815, 615313.929 1...",1.916527e+06
2,32.0,NaN,NaN,NaN,"POLYGON ((615313.929 1061218.352, 616577.954 1...",1.916526e+06
3,33.0,NaN,NaN,NaN,"POLYGON ((616577.954 1061214.942, 617841.978 1...",1.916525e+06
4,34.0,NaN,NaN,NaN,"POLYGON ((617841.978 1061211.582, 619106.002 1...",1.916524e+06
...,...,...,...,...,...,...
7492,14385.0,347087.0,481938.452248,1595796.0,"POLYGON ((677087.327 835215.728, 678360.439 83...",1.930088e+06
7493,14386.0,334139.0,341532.921286,1266317.0,"POLYGON ((678360.439 835214.839, 679633.551 83...",1.930087e+06
7494,14387.0,132737.0,176153.085928,344724.0,"POLYGON ((679633.551 835214.000, 680906.662 83...",1.930087e+06
7495,14388.0,11144.0,3110.576608,7495.0,"POLYGON ((680906.662 835213.213, 682179.774 83...",1.930087e+06


In [31]:
grid_final = grid_final.fillna(0)

In [32]:
grid_final

,id,ID,area,index_right,geometry,area_g
0,30.0,0.0,0.000000,0.0,"POLYGON ((612785.880 1061225.328, 614049.904 1...",1.916528e+06
1,31.0,0.0,0.000000,0.0,"POLYGON ((614049.904 1061221.815, 615313.929 1...",1.916527e+06
2,32.0,0.0,0.000000,0.0,"POLYGON ((615313.929 1061218.352, 616577.954 1...",1.916526e+06
3,33.0,0.0,0.000000,0.0,"POLYGON ((616577.954 1061214.942, 617841.978 1...",1.916525e+06
4,34.0,0.0,0.000000,0.0,"POLYGON ((617841.978 1061211.582, 619106.002 1...",1.916524e+06
...,...,...,...,...,...,...
7492,14385.0,347087.0,481938.452248,1595796.0,"POLYGON ((677087.327 835215.728, 678360.439 83...",1.930088e+06
7493,14386.0,334139.0,341532.921286,1266317.0,"POLYGON ((678360.439 835214.839, 679633.551 83...",1.930087e+06
7494,14387.0,132737.0,176153.085928,344724.0,"POLYGON ((679633.551 835214.000, 680906.662 83...",1.930087e+06
7495,14388.0,11144.0,3110.576608,7495.0,"POLYGON ((680906.662 835213.213, 682179.774 83...",1.930087e+06


In [50]:
grid_final["density"] = grid_final["area"]/ grid_final["area_g"]
grid_final = gpd.GeoDataFrame(grid_final)

In [51]:
grid_final.to_file('/Users/naoyamorishita/Documents/working/nightlight/data/bld_grid.geojson', driver='GeoJSON')  

## Calculating Public Housing Density
The operation is almost same, except for estimating the areas of public housings with nearest neighbor join.

In [55]:
bldph = ph.sjoin_nearest(building)
bldph[["geometry", "area"]]

,geometry,area
102270,POINT (650397.233 861223.677),2400.450210
136530,POINT (658707.829 861743.379),2767.218187
39278,POINT (650005.892 865528.232),1496.869476
187137,POINT (656520.193 866055.328),2312.329744
56296,POINT (679548.691 867033.110),1693.786653
...,...,...
130792,POINT (653470.733 969185.682),1292.680747
8784,POINT (654193.355 969191.534),1551.642477
149358,POINT (649708.190 970084.032),1773.167132
129172,POINT (634559.229 970358.975),1900.960186


In [58]:
bldph.to_file('/Users/naoyamorishita/Documents/working/nightlight/data/bldph.geojson', driver='GeoJSON')  